In [2]:
!pip install model-registry==0.2.15

Looking in indexes: http://nexus.apps.cluster-btrvw.btrvw.sandbox5316.opentlc.com/repository/pypi-internal/simple


In [3]:
from model_registry import ModelRegistry
from model_registry.utils import S3Params
from model_registry.exceptions import StoreError
import os

In [4]:
# Add your user name and cluster domain (apps.xxx) that are shared with you before

username = "fraud"
cluster_domain = "apps.cluster-btrvw.btrvw.sandbox5316.opentlc.com"

In [5]:
# Set up the model registry connection
model_registry_url = f"https://{username}-registry-rest.{cluster_domain}"
author_name = username

registry = ModelRegistry(server_address=model_registry_url, port=443, author=author_name, is_secure=False)

09:18:22.047 - model-registry:WARNING: Sourcing user token from default envvar: KF_PIPELINES_SA_TOKEN_PATH


In [6]:
# Model details we want to register
registered_model_name = "fraud"
version = "0.0.1"
model_path = "models/fraud/"

In [7]:
s3_upload_params = S3Params(
    bucket_name=os.environ.get('AWS_S3_BUCKET'),
    s3_prefix="models/fraud/",
)

try:
    registered_model = registry.upload_artifact_and_register_model(
        name=registered_model_name,
        model_files_path=model_path,
        model_format_name="onnx",
        author=username,
        model_format_version="1",
        version=version,
        description="data",
        metadata={
            "accuracy": 0.3,
            "license": "apache-2.0"
        },
        upload_params=s3_upload_params
    )

    print(f"'{registered_model_name}' version '{version}' has been registered here: https://rhods-dashboard-redhat-ods-applications.{cluster_domain}/modelRegistry/{username}-registry/registeredModels/1/versions/{registry.get_model_version(registered_model_name, version).id}/details")

except StoreError:
    rmver = registry.get_model_version(registered_model_name, version)
    print(f"Model and version already exists:\n{rmver}")

'fraud' version '0.0.1' has been registered here: https://rhods-dashboard-redhat-ods-applications.apps.cluster-btrvw.btrvw.sandbox5316.opentlc.com/modelRegistry/fraud-registry/registeredModels/1/versions/2/details


In [8]:
# Print the general info of registered model
model = registry.get_registered_model("fraud")
print("Registered Model:", model, "with ID", model.id)

Registered Model: id='1' description=None external_id=None create_time_since_epoch='1756545505439' last_update_time_since_epoch='1756545505439' custom_properties=None name='fraud' owner='fraud' state=<RegisteredModelState.LIVE: 'LIVE'> with ID 1


In [9]:
# Print the version info of registered model
version = registry.get_model_version("fraud", "0.0.1")
print("Model Version:", version, "with ID", version.id)

Model Version: id='2' description='data' external_id=None create_time_since_epoch='1756545505566' last_update_time_since_epoch='1756545505566' custom_properties={'accuracy': 0.3, 'license': 'apache-2.0'} name='0.0.1' author='fraud' state=<ModelVersionState.LIVE: 'LIVE'> with ID 2


In [10]:
# Print the artifact info of registered model
art = registry.get_model_artifact("fraud", "0.0.1")
print("Model Artifact:", art, "with ID", art.id)

Model Artifact: id='1' description=None external_id=None create_time_since_epoch='1756545505678' last_update_time_since_epoch='1756545505678' custom_properties=None name='fraud' uri='s3://mystorage/models/fraud?endpoint=https://minio-api-minio.apps.cluster-btrvw.btrvw.sandbox5316.opentlc.com&defaultRegion=none' state=<ArtifactState.UNKNOWN: 'UNKNOWN'> model_format_name='onnx' model_format_version='1' storage_key=None storage_path=None service_account_name=None model_source_kind=None model_source_class=None model_source_group=None model_source_id=None model_source_name=None with ID 1
